# Performance Optimization

Now that the Gibbs sampling algorithm and analysis function appear to be working correctly, I'd like to optimize. There are certain easy optimizations that will probably save at least a little time.

This is my starting point (running `time build/cnphylogeny data/PTX005/input`):

```
real    0m53.035s
user    0m52.956s
sys     0m0.002s
```

Note that I've already applied compiler optimizations, so any speedup from here will be my own doing.

## Stop Normalizing Probability Distributions

Currently, I normalize probabilities, like this:

```
double total = 0;
for (int i = 0; i <= max_copy_num; i++) total += probs[i];
for (int i = 0; i <= max_copy_num; i++) probs[i] /= total;
```

This wastes time and modifies `probs`, which is a potential bug waiting to happen. First step is to make the sampling algorithm account for `total` instead of doing a bunch of divisions. This provides some performance gain:

```
real    0m49.910s
user    0m49.844s
sys     0m0.002s
```

### Eliminate Extra Loop When Calculating Distribution Total

```
for (int i = 0; i <= max_copy_num; i++) total += probs[i];
```

This is not ideal, because in order to get the probabilities in the first place, we've already iterated over all of them. Hence, I put the total calculation outside the `sample()` function to eliminate another source of wasted time. This didn't really make a significant difference in performance (I could test further, but since I know that this method couldn't possibly be worse, I'm going to keep it):

```
real    0m49.397s
user    0m49.324s
sys     0m0.005s
```

### Calculate Phylogeny Score During Iteration

Another major slowdown is the fact that I'm calling `gibbs_node_analyze()` in between each iteration, which traverses the entire phylogeny in order to calculate its score. However, I should be able to do this calculation within the iteration function itself, since the probability (potential) for each bin must be calculated anyway. That's my next step...